In [2]:
# from bpnetlite import BPNet, ChromBPNet
from tangermeme.io import extract_loci
from tangermeme.predict import predict
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from models import BPExtractor
import data
import torch
import models
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.optim import Adam
from sklearn.metrics import mean_squared_error
import gzip
from torch_geometric.utils import scatter

import importlib

importlib.reload(data)
importlib.reload(models)

from data import sample_tpm_rows, remove_duplicate_edges, build_enhancer_graph, add_virtual_node, EnhancerGraphDataset,EnhancerGraphDatasetClassify
from models import VirtualNodeGNN, GeneExpressionGNN

import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/users/kobbad/anaconda3/envs/diff_model/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# check if cuda is available and device id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# check which device is being used
print(device)

cuda


In [4]:
peak_file = "/oak/stanford/groups/akundaje/kobbad/CS224W/data/ENCFF439EIO.bed.gz"
gene_data = "./data/gene_data.tsv"

gene_data_df = pd.read_csv(gene_data, sep="\t")
peaks = pd.read_csv(peak_file, sep="\t", header=None)
peaks = peaks.drop_duplicates(subset=peaks.columns[[0, 1, 2]])
peaks.head()

embeddings = np.load("./data/embeddings.npz")
embeddings = embeddings['embeddings']

In [5]:
# genes = sample_tpm_rows(gene_data_df, 10, percentile = 10)
# genes = genes[(genes.TPM <1000) & (genes.TPM > 0.5)]

gene_zero = gene_data_df[gene_data_df.TPM == 0]
gene_nonzero = gene_data_df[gene_data_df.TPM > 0]


gene_zero = gene_zero.sample(gene_nonzero.shape[0])

genes = pd.concat([gene_zero, gene_nonzero])

In [115]:
# train chroms are all chroms except chr1 chr8 and chr 10
train_chroms = [f"chr{x}" for x in range(2, 8)] + [f"chr{x}" for x in range(9, 23)]
val_chroms = ["chr8", "chr10"]
test_chroms = ["chr1"]

train_genes = genes[genes["seqname"].isin(train_chroms)]
val_genes = genes[genes["seqname"].isin(val_chroms)]
test_genes = genes[genes["seqname"].isin(test_chroms)]

len(train_genes), len(val_genes), len(test_genes)


(12862, 1213, 1350)

In [116]:
train_genes = train_genes.sample(10000)

train_genes.head(10000)

sum(train_genes["TPM"] == 0) / len(train_genes)

0.4926

In [117]:
sum(val_genes["TPM"] == 0) / len(val_genes)

0.5507007419620775

In [118]:
train_dataset = EnhancerGraphDatasetClassify(train_genes, peaks, embeddings, gene_window=50_000, enhancer_window=20_000)
val_dataset = EnhancerGraphDatasetClassify(val_genes, peaks, embeddings, gene_window=50_000, enhancer_window=20_000)

In [123]:
test_dataset = EnhancerGraphDatasetClassify(test_genes, peaks, embeddings, gene_window=50_000, enhancer_window=20_000)


In [119]:
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=8)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)


In [124]:
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8)

In [120]:
# Model, optimizer, and loss
input_dim = train_dataset[0].x.size(1)  # Feature dimension
hidden_dim = 128
output_dim = 1  # Predicting a single value (e.g., regression)

model = VirtualNodeGNN(input_dim, hidden_dim, output_dim, num_layers=1)
#model = GeneExpressionGNN(input_dim, hidden_dim, output_dim)

optimizer = Adam(model.parameters(), lr=0.001)

#loss_fn = torch.nn.MSELoss()  # Example for regression
loss_fn = torch.nn.BCEWithLogitsLoss()
model.to(device);

In [122]:
from tqdm import tqdm

# Training loop
train_loss_list = []
val_accuracy_list = []

for epoch in range(200):
    model.train()  # Set model to training mode
    total_loss = 0
    
    # Training loop
    for batch in tqdm(train_loader, total=len(train_loader)):
        optimizer.zero_grad()
        batch.to('cuda')  # Move batch to GPU if available
        
        # Forward pass
        out = model(batch)
        loss = loss_fn(out.squeeze(), batch.y.float().squeeze())  # Match dimensions
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Track training loss
        total_loss += loss.item()
        train_loss_list.append(loss.item())
    
    print(f"Epoch {epoch}, Training Loss: {total_loss / len(train_loader)}")
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():
        for val_batch in val_loader:  # Use validation loader
            val_batch.to('cuda')  # Move to GPU if available
            
            # Forward pass
            logits = model(val_batch)  # Get raw logits
            probs = torch.sigmoid(logits)  # Convert logits to probabilities
            predictions = (probs > 0.5).float()  # Threshold probabilities
            
            # Calculate accuracy
            correct += (predictions.squeeze() == val_batch.y).sum().item()
            total += val_batch.y.size(0)
    
    # Compute accuracy
    accuracy = correct / total
    val_accuracy_list.append(accuracy)
    print(f"Epoch {epoch}, Validation Accuracy: {accuracy:.4f}")


100%|██████████| 40/40 [00:20<00:00,  1.91it/s]

Epoch 0, Training Loss: 0.6355808436870575


Epoch 0, Validation Accuracy: 0.5894


100%|██████████| 40/40 [00:20<00:00,  1.91it/s]

Epoch 1, Training Loss: 0.6305963009595871


...

100%|██████████| 40/40 [00:20<00:00,  1.92it/s]

Epoch 103, Training Loss: 0.5695682510733604


Epoch 103, Validation Accuracy: 0.6241


 40%|████      | 16/40 [00:10<00:16,  1.47it/s]


KeyboardInterrupt: 

In [125]:
# evaluate on test set
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for test_batch in test_loader:
        test_batch.to('cuda')
        
        # Forward pass
        logits = model(test_batch)  # Get raw logits
        probs = torch.sigmoid(logits)  # Convert logits to probabilities
        predictions = (probs > 0.5).float()  # Threshold probabilities
        
        # Calculate accuracy
        correct += (predictions.squeeze() == test_batch.y).sum().item()
        total += test_batch.y.size(0)

In [126]:
accuracy = correct / total

print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5963


In [127]:
# save model
torch.save(model.state_dict(), "./models/model_1_layer.pth")

In [129]:
sum(test_genes.TPM == 0) / len(test_genes)

0.46444444444444444

Testing deeper model

In [130]:
# Model, optimizer, and loss
input_dim = train_dataset[0].x.size(1)  # Feature dimension
hidden_dim = 128
output_dim = 1  # Predicting a single value (e.g., regression)

model = VirtualNodeGNN(input_dim, hidden_dim, output_dim, num_layers=2)
#model = GeneExpressionGNN(input_dim, hidden_dim, output_dim)

optimizer = Adam(model.parameters(), lr=0.001)

#loss_fn = torch.nn.MSELoss()  # Example for regression
loss_fn = torch.nn.BCEWithLogitsLoss()
model.to(device);

In [132]:
from tqdm import tqdm

# Training loop
train_loss_list = []
val_accuracy_list = []

for epoch in range(200):
    model.train()  # Set model to training mode
    total_loss = 0
    
    # Training loop
    for batch in tqdm(train_loader, total=len(train_loader)):
        optimizer.zero_grad()
        batch.to('cuda')  # Move batch to GPU if available
        
        # Forward pass
        out = model(batch)
        loss = loss_fn(out.squeeze(), batch.y.float().squeeze())  # Match dimensions
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Track training loss
        total_loss += loss.item()
        train_loss_list.append(loss.item())
    
    print(f"Epoch {epoch}, Training Loss: {total_loss / len(train_loader)}")
    
    # Validation loop
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():
        for val_batch in val_loader:  # Use validation loader
            val_batch.to('cuda')  # Move to GPU if available
            
            # Forward pass
            logits = model(val_batch)  # Get raw logits
            probs = torch.sigmoid(logits)  # Convert logits to probabilities
            predictions = (probs > 0.5).float()  # Threshold probabilities
            
            # Calculate accuracy
            correct += (predictions.squeeze() == val_batch.y).sum().item()
            total += val_batch.y.size(0)
    
    # Compute accuracy
    accuracy = correct / total
    val_accuracy_list.append(accuracy)
    print(f"Epoch {epoch}, Validation Accuracy: {accuracy:.4f}")


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

Epoch 0, Training Loss: 0.6501533821225166


Epoch 0, Validation Accuracy: 0.5886


100%|██████████| 40/40 [00:23<00:00,  1.69it/s]

Epoch 1, Training Loss: 0.6468512430787087


...

Epoch 198, Training Loss: 0.5288216210901737


Epoch 198, Validation Accuracy: 0.6249


100%|██████████| 40/40 [00:22<00:00,  1.74it/s]

Epoch 199, Training Loss: 0.5328224480152131


Epoch 199, Validation Accuracy: 0.6397


In [77]:
preds = model(next(iter(train_loader)).to('cuda')).cpu().detach().numpy()
actual = next(iter(train_loader)).y.numpy()

preds, actual

(array([[ 4.4399805],
        [-1.6007725],
        [ 1.28059  ],
        [-1.6427449],
        [ 1.4343225],
        [ 6.2365437],
        [ 1.2699493],
        [ 2.7057095],
        [-1.8779684],
        [ 0.7467816]], dtype=float32),
 array([0., 0., 1., 1., 1., 0., 1., 0., 1., 0.], dtype=float32))

In [16]:
# clear cuda cache
torch.cuda.empty_cache()
del model

In [ ]:
for i in range(len(train_dataset)):
    try:
        train_dataset[i]
    except:
        print(i)
        break

In [14]:
_, chrom, start, end, label = train_dataset.dataframes.iloc[72]
label = np.log1p(label)
adj_peaks = train_dataset.peaks[(train_dataset.peaks[1] > start - train_dataset.gene_window) & (train_dataset.peaks[2] < end + train_dataset.gene_window) & (train_dataset.peaks[0] == chrom)]

In [15]:
counter = 0
edge_index = []

embedding_index = []

for row in adj_peaks.iterrows():
    embedding_index.append(row[0])
    second_counter = 0

    _, start, end, *_ = row[1]

    for second_row in adj_peaks.iterrows():
        _, second_start, second_end, *_ = second_row[1]

        if abs(start - second_start) < train_dataset.enhancer_window and abs(end - second_end) < train_dataset.enhancer_window:
            edge_index.append([counter, second_counter])

        second_counter += 1

    counter += 1

# remove self loops
edge_index = [edge for edge in edge_index if edge[0] != edge[1]]

edge_index = torch.tensor(edge_index, dtype=torch.long).T

In [22]:
edge_index.shape[0]

0